In [2]:
"""
###
#this csv include wrongly definite SMILE data and dgl library cannot work properly with them (eg include {,} etc...)
##
import pandas as pd
df=pd.read_csv('data/train_validation_cardio_tox_data.csv')
df.head()
"""

"\n###\n#this csv include wrongly definite SMILE data and dgl library cannot work properly with them (eg include {,} etc...)\n##\nimport pandas as pd\ndf=pd.read_csv('data/train_validation_cardio_tox_data.csv')\ndf.head()\n"

In [3]:
#import data to dataframe from csv
import pandas as pd
df_1=pd.read_csv('data/external_test_set_neg.csv')
df_2=pd.read_csv('data/external_test_set_new.csv')
df_3=pd.read_csv('data/external_test_set_pos.csv')
frames = [df_1, df_2, df_3]
df = pd.concat(frames)

In [4]:
# read data with smiles
smiles_postive= []
smiles_negative = []
acitivity = df['ACTIVITY'].tolist()
smiles = df['smiles'].tolist()
for i in range (0, len(smiles), 1):
    if smiles[i]!='{' or '}':
        if acitivity[i] == 0:
            smiles_negative.append(smiles[i])
        if acitivity[i] == 1:
            smiles_postive.append(smiles[i])



In [5]:
import dgl
from dgl.data import DGLDataset
import torch
import torch as th
import os
from ogb.utils.features import (allowable_features, atom_to_feature_vector,
 bond_to_feature_vector, atom_feature_vector_to_dict, bond_feature_vector_to_dict)
from rdkit import Chem
import numpy as np

In [6]:
def smiles2graph(smiles_string):
    """
    Converts SMILES string to graph Data object
    :input: SMILES string (str)
    :return: graph object
    """

    mol = Chem.MolFromSmiles(smiles_string)

    A = Chem.GetAdjacencyMatrix(mol)
    A = np.asmatrix(A)
    nnodes=len(A)
    nz = np.nonzero(A)
    edge_list=[]
    src=[]
    dst=[]

    for i in range(nz[0].shape[0]):
      src.append(nz[0][i])
      dst.append(nz[1][i])

    u, v = src, dst
    g = dgl.graph((u, v))
    bg=dgl.to_bidirected(g)

    return bg

In [7]:
def feat_vec(smiles_string):
    """
    Returns atom features for a molecule given a smiles string
    """
    # atoms
    mol = Chem.MolFromSmiles(smiles_string)
    atom_features_list = []
    for atom in mol.GetAtoms():
        atom_features_list.append(atom_to_feature_vector(atom))
    x = np.array(atom_features_list, dtype = np.int64)
    return x

In [8]:
# This block makes a list of graphs
lista_mols_positive=smiles_postive

j=0
graphs_positive=[]
execptions_positive=[]
for mol in lista_mols_positive:

  g_mol=smiles2graph(mol)

  try:
    g_mol.ndata['feat']=torch.tensor(feat_vec(mol))
  except:
    execptions_positive.append(j)


  graphs_positive.append(g_mol)
  j+=1

In [9]:
#Positive =>Toxic
labels_positive=[]

for item in lista_mols_positive:
    labels_positive.append(1)


In [10]:
#Negative
lista_mols_negative=smiles_negative

j=0
graphs_negative=[]
execptions_negative=[]
for mol in lista_mols_positive:

  g_mol=smiles2graph(mol)

  try:
    g_mol.ndata['feat']=torch.tensor(feat_vec(mol))
  except:
    execptions_negative.append(j)


  graphs_negative.append(g_mol)
  j+=1

In [11]:
#Negative =>Non-Toxic
labels_negative=[]

for item in lista_mols_negative:
    labels_negative.append(0)

In [12]:
graphs = graphs_positive + graphs_negative
labels = labels_positive + labels_negative
execptions = execptions_negative + execptions_positive
graphs = graphs[:len(labels)]


In [13]:
# Some smiles are not well processed, so they are droped
ii=0
for i in execptions:
  graphs.pop(i-ii)
  labels.pop(i-ii)
  ii+=1

In [14]:
i=0
for grap in graphs:

  try:
    grap.ndata['feat']
  except:
    print(i)
  i+=1

In [15]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

In [16]:
class SyntheticDataset(DGLDataset):
    def __init__(self):
        super().__init__(name='synthetic')

    def process(self):
        #edges = pd.read_csv('./graph_edges.csv')
        #properties = pd.read_csv('./graph_properties.csv')
        self.graphs = graphs
        self.labels = torch.LongTensor(labels)

    def __getitem__(self, i):
        return self.graphs[i], self.labels[i]

    def __len__(self):
        return len(self.graphs)

dataset = SyntheticDataset()
#graph, label = dataset[0]
#print(graph, label)

In [17]:
from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler

num_examples = len(dataset)
num_train = int(num_examples * 0.8)

train_sampler = SubsetRandomSampler(torch.arange(num_train))
test_sampler = SubsetRandomSampler(torch.arange(num_train, num_examples))

train_dataloader = GraphDataLoader(
    dataset, sampler=train_sampler, batch_size=5, drop_last=False)
test_dataloader = GraphDataLoader(
    dataset, sampler=test_sampler, batch_size=5, drop_last=False)

In [18]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

In [19]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        g.ndata['h'] = h
        return dgl.mean_nodes(g, 'h')

In [20]:
# Create the model with given dimensions
model = GCN(9, 8, 2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


model.train()
for epoch in range(50):
    epoch_loss = 0
    epoch_accuracy = 0
    for batched_graph, labels in train_dataloader:
        pred = model(batched_graph, batched_graph.ndata['feat'].float())
        #print(pred,labels)
        loss = F.cross_entropy(pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = ((pred.argmax(dim=1) == labels).float().mean())
        epoch_accuracy += acc/len(train_dataloader)
        epoch_loss += loss/len(train_dataloader)
    print('Epoch : {}, train accuracy : {}, train loss : {}'.format(epoch+1, epoch_accuracy,epoch_loss))

num_correct = 0
num_tests = 0
for batched_graph, labels in test_dataloader:
    pred = model(batched_graph, batched_graph.ndata['feat'].float())

    num_correct += (pred.argmax(1) == labels).sum().item()
    num_tests += len(labels)



Epoch : 1, train accuracy : 0.5249999761581421, train loss : 0.9571995139122009
Epoch : 2, train accuracy : 0.6250001192092896, train loss : 0.6667819619178772
Epoch : 3, train accuracy : 0.625, train loss : 0.6718767881393433
Epoch : 4, train accuracy : 0.6250000596046448, train loss : 0.6636510491371155
Epoch : 5, train accuracy : 0.6166667342185974, train loss : 0.660469114780426
Epoch : 6, train accuracy : 0.6166666150093079, train loss : 0.6615277528762817
Epoch : 7, train accuracy : 0.6333333253860474, train loss : 0.6610697507858276
Epoch : 8, train accuracy : 0.6333333849906921, train loss : 0.6636439561843872
Epoch : 9, train accuracy : 0.6333334445953369, train loss : 0.6662455201148987
Epoch : 10, train accuracy : 0.6166666746139526, train loss : 0.6625040769577026
Epoch : 11, train accuracy : 0.625, train loss : 0.664335310459137
Epoch : 12, train accuracy : 0.6333333253860474, train loss : 0.6676678657531738
Epoch : 13, train accuracy : 0.6000000834465027, train loss : 0.6

In [21]:
def transform_smile(string):
    graph=smiles2graph(string)
    graph.ndata['feat']=torch.tensor(feat_vec(string))
    return graph



def get_prediction(string)->int:
 foo = transform_smile(string)
 pred = model(foo, foo.ndata['feat'].float())
 result = int(pred.argmax(1))
 return result

def answear(smile):
    value = get_prediction(smile)
    if value == 1:
        result = 'Toxic'
    if value == 0:
        result = 'Non-Toxic'

    return result

#Clc1ccc(cc1)C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl
#'COc1ccc(C(=O)Nc2ccccc2CCC2CCCCN2C)cc1'
random_smile = 'CCN(CC)CCCN(c1ccccc1)C1CCc2ccccc21'
print(answear(random_smile))





Toxic


In [22]:
# For this notebook to run with updated APIs, we need torch 1.12+ and torchvision 0.13+
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

torch version: 1.13.1
torchvision version: 0.14.1


In [297]:
torch.save(model.state_dict(), "./  GraphNN.pt")